In [1]:
%cd ../
%pwd

D:\Projects\IndiaTB\IndTB_ppa


'D:\\Projects\\IndiaTB\\IndTB_ppa'

In [2]:
import numpy as np
import numpy.random as rd
from scipy import linalg
import pandas as pd
import json
from pydantic import BaseModel, Field

In [3]:
dat = pd.read_csv('data/cascade/d_cascade_2019.csv')
dat = {row['State']: row for _, row in dat.iterrows()}
d = dat['India']
d

Region              India
State               India
N              1347720000
PrevUt         290.745488
PrevTx         190.854947
Pr_Pub_CSI       0.482818
Pr_Pub_CSI0      0.614003
Pr_Pub_CSI1      0.606058
Pr_Asym          0.630429
Pr_NotAware      0.212551
Pr_NotCS         0.031728
Pr_NotDet        0.125292
Pop            1357660000
CNR_pub           0.00127
CNR_eng            0.0005
TxI_pub              0.94
TxI_eng              0.95
TxSucc_pub       0.903226
TxSucc_eng       0.910256
TxDead_pub       0.043011
TxDead_eng       0.025641
TxLTFU_pub       0.053763
TxLTFU_eng       0.064103
Name: 0, dtype: object

In [4]:
with open('out/rate_valid/Rates.json', 'r') as f:
    rates = json.load(f)

In [5]:
def r_inc():
    return rd.uniform(0.00132, 0.00266)

In [6]:
class Cascade(BaseModel):
    R_SelfCure: float
    R_Die_Asym: float
    R_Die_Sym: float
    R_Die_Tx: np.ndarray = Field(default_factory=lambda: np.zeros(3))

    R_Onset: float
    R_Aware: float
    R_CSI: float
    R_ReCSI: float

    P_Entry: np.ndarray = Field(default_factory=lambda: np.zeros(3))
    P_Tr: np.ndarray = Field(default_factory=lambda: np.zeros((3, 3)))
    P_Dx0: np.ndarray = Field(default_factory=lambda: np.zeros(3))
    P_Dx1: np.ndarray = Field(default_factory=lambda: np.zeros((3, 3)))

    P_TxI: np.ndarray = Field(default_factory=lambda: np.zeros(3))

    R_Succ_Tx: np.ndarray = Field(default_factory=lambda: np.zeros(3))
    R_LTFU_Tx: np.ndarray = Field(default_factory=lambda: np.zeros(3))

    PPV: np.ndarray = Field(default_factory=lambda: np.zeros(3))

    class Config:
        arbitrary_types_allowed = True
        
    @staticmethod
    def parse_json(rs):
        rs = {k: (np.array(v) if isinstance(v, list) else v) for k, v in rs.items()}
        return Cascade.parse_obj(rs)
        
    def match_targets(self):
        inc = r_inc()

        p_a = inc / (self.R_Onset + self.R_Die_Asym + self.R_SelfCure)
        p_s = (self.R_Onset * p_a) / (self.R_Aware + self.R_Die_Sym + self.R_SelfCure)
        p_c = (self.R_Aware * p_s) / (self.R_CSI + self.R_Die_Sym + self.R_SelfCure)


        cs0 = self.R_CSI * p_c * self.P_Entry
        fn0 = cs0 * (1 - self.P_Dx0)
        det0 = cs0 * self.P_Dx0

        mu = self.R_ReCSI + self.R_Die_Sym + self.R_SelfCure

        p_e_hs = linalg.solve(self.R_ReCSI * (self.P_Tr * (1 - self.P_Dx1)).T - np.eye(3) * mu, -fn0)
        p_e = p_e_hs.sum()

        det1 = (self.R_ReCSI * self.P_Tr * self.P_Dx1 * p_e_hs).sum(0)
        det = det0 + det1

        det_all = det / self.PPV 
        tx = det_all * self.P_TxI
        
        dur = 1 / (self.R_LTFU_Tx + self.R_Succ_Tx + self.R_Die_Tx)

        return {
            'IncR': inc,
            'PrevAsym': p_a,
            'PrevNotAware': p_s,
            'PrevNotCS': p_c,
            'PrevNotDet': p_e,
            'Prev': p_a + p_s + p_c + p_e,
            'CNR_Pub': det_all[0],
            'CNR_Eng': det_all[1],
            'Det_Pri': det[2],
            'PrTxi_pri': self.P_TxI[2],
            'Dur_Pub': dur[0],
            'Dur_Eng': dur[1]
        }


In [7]:
Css = [Cascade.parse_json(rs) for rs in rates]

In [8]:
mss = list()

for i, cs in enumerate(Css):
    tar = cs.match_targets()
    tar['Key'] = i
    mss.append(tar)
    
mss = pd.DataFrame(mss).set_index('Key')
mss

,IncR,PrevAsym,PrevNotAware,PrevNotCS,PrevNotDet,Prev,CNR_Pub,CNR_Eng,Det_Pri,PrTxi_pri,Dur_Pub,Dur_Eng
Key,,,,,,,,,,,,
0,0.001881,0.001787,0.000602,0.000090,0.000355,0.002834,0.001084,0.000446,0.000049,0.623320,0.451613,0.455128
1,0.002008,0.001907,0.000643,0.000096,0.000379,0.003024,0.001102,0.000556,0.000043,0.739854,0.451613,0.455128
2,0.001584,0.001505,0.000507,0.000076,0.000299,0.002387,0.000903,0.000366,0.000040,0.704602,0.451613,0.455128
3,0.002316,0.002200,0.000742,0.000111,0.000437,0.003490,0.001124,0.000568,0.000004,0.702721,0.451613,0.455128
4,0.001968,0.001869,0.000630,0.000094,0.000371,0.002965,0.001039,0.000492,0.000009,0.547285,0.451613,0.455128
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.001488,0.001414,0.000477,0.000071,0.000281,0.002242,0.000789,0.000424,0.000016,0.626420,0.451613,0.455128
296,0.002367,0.002248,0.000758,0.000113,0.000447,0.003566,0.001278,0.000583,0.000019,0.875824,0.451613,0.455128
297,0.001815,0.001724,0.000581,0.000087,0.000343,0.002735,0.000988,0.000450,0.000025,0.954970,0.451613,0.455128


In [9]:
mss.to_csv('out/rate_valid/valid.csv')

In [10]:
rates[0]

{'R_Die_Asym': 0.01741473767592514,
 'R_Die_Sym': 0.16138198815724747,
 'R_Die_Tx': [0.09523809523809515, 0.05633802816901399, 0.07785670990708789],
 'R_SelfCure': 0.13513361720993325,
 'R_Onset': 0.9004023453244097,
 'R_Aware': 2.374094274499213,
 'R_CSI': 15.607825898609267,
 'R_ReCSI': 4.574685953595595,
 'P_Entry': [0.4828184936896596, 0.42015781865425056, 0.09702368765608987],
 'P_Tr': [[0.5539759944190364, 0.3623495251972085, 0.08367448038375509],
  [0.6665871456145235, 0.27086432113415315, 0.0625485332513234],
  [0.6665871456145235, 0.27086432113415315, 0.0625485332513234]],
 'P_Dx0': [0.0784313725490196, 0.0784313725490196, 0.23021925366210175],
 'P_Dx1': [[0.7481203007518797, 0.7481203007518797, 0.912567249468275],
  [0.7481203007518797, 0.7481203007518797, 0.912567249468275],
  [0.14068441064638784, 0.14068441064638784, 0.3652063512662714]],
 'P_TxI': [0.94, 0.95, 0.6233202915619193],
 'PPV': [0.85, 0.85, 0.43771079800666723],
 'R_Succ_Tx': [2.0, 2.0, 2.7639132017016244],
 'R